<a href="https://colab.research.google.com/github/gauravpathak91/NavTab/blob/master/GAN_and_Extra_tree_awid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==1.4.0

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 1.x selected.
Found GPU at: /device:GPU:0


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
data = pd.read_csv("/content/drive/My Drive/awid/awid_cls_train_clean_int_vals.csv")

In [ ]:
data = data.drop(columns="Unnamed: 0")

In [ ]:
data.head()

,3,4,5,6,7,8,37,46,47,49,50,60,63,65,66,67,68,69,70,71,72,74,75,154
0,1.393661e+09,0.000000,0.000000,0.000000,261,261,2101598203,1.0,2437,1,0,-47,8,0,8,0,0,0,0,0,0,0,281474976710655,normal
1,1.393661e+09,0.024271,0.024271,0.024271,185,185,2101623071,1.0,2437,1,0,-47,8,0,8,0,0,0,0,0,0,0,281474976710655,normal
2,1.393661e+09,0.001631,0.001631,0.025902,185,185,2101624680,1.0,2437,1,0,-64,8,0,8,0,0,0,0,0,0,0,281474976710655,normal
3,1.393661e+09,0.055325,0.055325,0.081227,159,159,2101680214,1.0,2437,1,0,-32,8,0,8,0,0,0,0,0,0,0,281474976710655,normal
4,1.393661e+09,0.000415,0.000415,0.081642,54,54,2101681730,48.0,2437,0,1,-21,36,2,4,1,0,0,0,0,0,44,44833259836374,normal


In [ ]:
data_normal = data[data["154"]=="normal"]

In [ ]:
data["154"].value_counts()

normal           1631218
injection          65379
impersonation      48522
flooding           48484
Name: 154, dtype: int64

In [ ]:
data_normal =data_normal.drop(columns=["3", "5","7","47","50","67","70","37"])

In [ ]:
data_normal.shape

(1631218, 16)

In [ ]:
X = data_normal.drop(columns="154")

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_norm = MinMaxScaler().fit(X)
X_train_norm = X_norm.transform(X)

In [ ]:
norm_dataframe = pd.DataFrame(X_train_norm)

In [ ]:
learning_rate = 0.0002
batch_size = 1000
epochs = 1000

data_dim = 15
gen_hidden_dim = 100
gen_hidden1_dim = 100
disc_hidden_dim = 100
disc_hidden1_dim = 100
z_noise_dim = 15

In [ ]:
def xavier_init(shape):
  return tf.random_normal(shape=shape, stddev=1./tf.sqrt(shape[0]/2.))

In [ ]:
weights = {"disc_H":tf.Variable(xavier_init([data_dim, disc_hidden_dim])),
           "disc_H1":tf.Variable(xavier_init([disc_hidden_dim, disc_hidden1_dim])),
           "disc_final":tf.Variable(xavier_init([disc_hidden1_dim,1])),
           "gen_H":tf.Variable(xavier_init([z_noise_dim, gen_hidden_dim])),
           "gen_H1":tf.Variable(xavier_init([gen_hidden_dim, gen_hidden1_dim])),
           "gen_final":tf.Variable(xavier_init([gen_hidden1_dim, data_dim]))
          }

bias = {"disc_H":tf.Variable(xavier_init([disc_hidden_dim])),
        "disc_H1":tf.Variable(xavier_init([disc_hidden1_dim])),
        "disc_final":tf.Variable(xavier_init([1])),
        "gen_H":tf.Variable(xavier_init([gen_hidden_dim])),
        "gen_H1":tf.Variable(xavier_init([gen_hidden1_dim])),
        "gen_final":tf.Variable(xavier_init([data_dim]))}

In [ ]:
def Generator(x):
    hidden_layer = tf.matmul(x, weights['gen_H'])
    hidden_layer = tf.add(hidden_layer, bias['gen_H'])
    hidden_layer = tf.nn.relu(hidden_layer)
    hidden1_layer = tf.matmul(hidden_layer, weights['gen_H1'])
    hidden1_layer = tf.add(hidden1_layer, bias['gen_H1'])
    hidden1_layer = tf.nn.relu(hidden1_layer)
    out_layer = tf.matmul(hidden1_layer, weights['gen_final'])
    out_layer = tf.add(out_layer, bias['gen_final'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer


# Discriminator
def Discriminator(x):
    hidden_layer = tf.matmul(x, weights['disc_H'])
    hidden_layer = tf.add(hidden_layer, bias['disc_H'])
    hidden_layer = tf.nn.relu(hidden_layer)
    hidden1_layer = tf.matmul(hidden_layer, weights['disc_H1'])
    hidden1_layer = tf.add(hidden1_layer, bias['disc_H1'])
    hidden1_layer = tf.nn.relu(hidden1_layer)
    out_layer = tf.matmul(hidden1_layer, weights['disc_final'])
    out_layer1 = tf.add(out_layer, bias['disc_final'])
    out_layer = tf.nn.sigmoid(out_layer1)
    return out_layer1, out_layer

In [ ]:
z_input = tf.placeholder(tf.float32, shape =[None, z_noise_dim], name = "input_noise")
x_input = tf.placeholder(tf.float32, shape =[None, data_dim], name = "real_input")

In [ ]:
with tf.name_scope("Generator") as scope:
  output_Gen = Generator(z_input) #g(x)

with tf.name_scope("Descriminator") as scope:
  real_output1_Disc, real_output_Disc = Discriminator(x_input) #d(x)
  fake_output1_Disc, fake_output_Disc = Discriminator(output_Gen) #d(g(x))

In [ ]:
with tf.name_scope("Discriminator_Loss") as scope:
  Discriminator_Loss = -tf.reduce_mean(tf.log(real_output_Disc+0.0001) + tf.log(1.-fake_output_Disc+0.0001))

with tf.name_scope("Generator_Loss") as scope:
  Generator_Loss = -tf.reduce_mean(tf.log(fake_output_Disc+0.0001))

Disc_loss_total = tf.summary.scalar("Disc_Total_loss", Discriminator_Loss)
Gen_loss_total = tf.summary.scalar("Gen_Total_loss", Generator_Loss)

In [ ]:
Generator_var = [weights["gen_H"], weights["gen_final"], bias["gen_H"], bias["gen_final"]]
Descriminator_var = [weights["disc_H"], weights["disc_final"], bias["disc_H"], bias["disc_final"]]

In [ ]:
with tf.name_scope("Optimizer_Discriminator") as scope:
  Discriminator_optimize = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(Discriminator_Loss, var_list = Descriminator_var)

with tf.name_scope("Optimizer_Generator") as scope:
  Generator_optimize = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(Generator_Loss, var_list = Generator_var)

In [ ]:
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
writer = tf.summary.FileWriter("./logs", sess.graph)

for epoch in range(epochs):
  for i in range(0, len(norm_dataframe), batch_size):
    batch = norm_dataframe[i:i+batch_size]
    X_batch = batch
    z_noise = np.random.uniform(-1., 1., size = [len(batch), z_noise_dim])
    _, Disc_loss_epoch = sess.run([Discriminator_optimize, Discriminator_Loss], feed_dict = {x_input:X_batch, z_input:z_noise})
    _, Gen_loss_epoch = sess.run([Generator_optimize, Generator_Loss], feed_dict = {z_input:z_noise})

    #summary_Disc_Loss = sess.run(Disc_loss_total, feed_dict = {x_input:X_batch, z_input:z_noise})
    #writer.add_summary(summary_Disc_Loss, epoch)

    #summary_Gen_Loss = sess.run(Gen_loss_total, feed_dict = {z_input:z_noise})
    #writer.add_summary(summary_Gen_Loss, epoch)

  if epoch % 100 ==0:
    print("steps: {0} : Generator Loss: {1}, Descriminator Loss: {2}".format(epoch, Gen_loss_epoch, Disc_loss_epoch))

steps: 0 : Generator Loss: 0.7927548885345459, Descriminator Loss: 1.0789471864700317
steps: 100 : Generator Loss: 1.076942801475525, Descriminator Loss: 1.2360808849334717
steps: 200 : Generator Loss: 1.070707082748413, Descriminator Loss: 1.1237000226974487
steps: 300 : Generator Loss: 1.3215184211730957, Descriminator Loss: 1.1224159002304077
steps: 400 : Generator Loss: 1.2164217233657837, Descriminator Loss: 1.159535527229309
steps: 500 : Generator Loss: 2.0023632049560547, Descriminator Loss: 0.9341744184494019
steps: 600 : Generator Loss: 1.6159111261367798, Descriminator Loss: 1.1019465923309326
steps: 700 : Generator Loss: 1.5397357940673828, Descriminator Loss: 1.0835018157958984
steps: 800 : Generator Loss: 1.8236333131790161, Descriminator Loss: 0.8896986842155457
steps: 900 : Generator Loss: 1.8597121238708496, Descriminator Loss: 0.9204854965209961


In [ ]:
Z_noise = np.random.uniform(-1., 1., size= [100000, z_noise_dim])
g = sess.run(output_Gen, feed_dict={z_input:Z_noise})

In [ ]:
data_gan = X_norm.inverse_transform(g)

In [ ]:
pd.DataFrame(data_gan).to_csv("/content/drive/My Drive/awid/gan_data.csv")

In [ ]:
X_ml = data.drop(columns =["3", "5","7","47","50","67","70","37","154"])
y_ml = data["154"]=="normal"
y_ml = y_ml.astype("int")

In [ ]:
X_ml.shape

(1793603, 15)

In [ ]:
from sklearn.preprocessing import RobustScaler
X_ml_norm = RobustScaler().fit(X_ml)
X_ml_train_norm = X_norm.transform(X_ml)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
sm = RandomUnderSampler(random_state=10)
X_train_res, y_train_res = sm.fit_resample(X_ml_train_norm, y_ml)

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
#clf_random_forest = RandomForestClassifier(n_estimators=10, max_depth=50, random_state=0)
clf_extra_tree = ExtraTreesClassifier(n_estimators=10, max_depth=25, random_state=0)

In [ ]:
clf_extra_tree.fit(X_train_res, y_train_res)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=25, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=0, verbose=0,
                     warm_start=False)

In [ ]:
data_gan_datafram = pd.DataFrame(data_gan)

In [ ]:
data_gan_trans = X_ml_norm.transform(data_gan)

In [ ]:
prediction_norm = clf_extra_tree.predict(data_gan_trans)

In [ ]:
pred_dataframe = pd.DataFrame(prediction_norm)

In [ ]:
 pred_dataframe[0].value_counts()

1    88597
0    11403
Name: 0, dtype: int64